Hey, ORville here 👋

I'm a logistics manager overseeing the allocation of tasks to employees in our company, FurnitORe.

Right now we have 100 tasks that need to be completed, and we also have 100 employees available to handle them.

The catch is that assigning a task to an employee has a cost 💸.

These costs vary depending on the difficulty of the task, the expertise of the employee, and other factors.

I need your help to figure out the most cost-effective way to assign these tasks to employees.

You can assume that each task is done by just one employee, and one employee is assigned to just one task.

Can you help me solve this problem?

## Solution modeled with binary variables

**Goal**

Minimal cost allocation of workers to tasks

$$ \min \sum_{w,t}c_{wt}x_{wt} $$

**Decisions**:

$x_{wt}\in\{0,1\}$ for worker $w$ allocated for task $t$


**Data:**

Set of workers $W = \{0..99\}$

Set of tasks $T = \{0..99\}$

Assignment Cost Matrix $c_{wt}$ for worker $w$ on task $t$



**Constraints**

| **Description** | **Expression** |
| --- | --- |
| Each employee is assigned exactly one task | $\sum_t x_{wt}=1$ |
| Each task is completed by exactly one employee | $\sum_w x_{wt}=1$ |

In [26]:
def get_instance(input_filename):
    with open(input_filename, "r") as file:
        data = file.read()

    # Split the data into lines
    lines = data.strip().split('\n')
    l = 0

    # Skip comment lines
    while lines[l].startswith('#') or lines[l] == '':
        l += 1
    
    # get metadata parameters (always first line)
    metadata = map(int, lines[l].split())
    data = []
    l += 1

    # Process each line
    while l < len(lines):
        data.append(list(map(int, lines[l].split())))
        l += 1

    return *metadata, data

def parse_to_zero_index(data, indexes):
    for i in range(len(data)):
        for idx in indexes:
            data[i][idx] -= 1
    return

In [27]:
def correct_format(data):
    for i in range(100):
        for _ in range(7):
            data[i].extend(data[i+1])
            data.pop(i+1)
    return

In [28]:
num_tasks, assigment_costs = get_instance("data/tasks_employees.txt")
num_workers = num_tasks
correct_format(assigment_costs)

# Print the extracted data
print(f"Number of tasks: {num_tasks}")
print(f"Number of workers: {num_workers}")
print(f"Assignment Cost matrix: {len(assigment_costs), len(assigment_costs[0])}")

Number of tasks: 100
Number of workers: 100
Assignment Cost matrix: (100, 100)


In [29]:
from ortools.sat.python import cp_model

model = cp_model.CpModel()
solver = cp_model.CpSolver()

In [30]:
x = [[None] * num_workers for _ in range(num_tasks)]

for t in range(num_tasks):
    for w in range(num_workers):
        x[t][w] = model.NewBoolVar(f'task_{t}_to_worker_{w}')

In [31]:
for t in range(num_tasks):
    model.Add(sum(x[t]) == 1)

In [32]:
for x_w in zip(*x):
    model.Add(sum(x_w) == 1)

In [34]:
# Specify the type of problem. In this case, we want to minimize the objective function
solver.parameters.num_search_workers = 8
solver.parameters.max_time_in_seconds = 120
model.Minimize(
    sum(assigment_costs[t][w] * x[t][w] 
        for t in range(num_tasks) 
        for w in range(num_workers)
    )
)

In [35]:
# Call the solver method to find the optimal solution
callback = cp_model.ObjectiveSolutionPrinter()
or_status = solver.SolveWithSolutionCallback(model, callback)
status = solver.StatusName(or_status)

if status in ["OPTIMAL", "FEASIBLE"]:
    print(f'Solution: Total cost of worker\'s payment = {solver.ObjectiveValue()}')
else:
    print('A solution could not be found, check the problem specification')

Solution 0, time = 0.68 s, objective = 5246
Solution 1, time = 0.84 s, objective = 3345
Solution 2, time = 0.89 s, objective = 305
Solution: Total cost of worker's payment = 305.0


In [87]:
solution1 = set()
for t in range(num_tasks):
    for w in range(num_workers):
        if solver.Value(x[t][w]) == 1:
            solution1.add((t, w))
# [solver.Value(x[s][v]) for v in range(num_cities) if x[s][v] is not None]

In [88]:
sum(assigment_costs[t][w] for t, w in solution1)

305

Greedy Approach

In [101]:
from collections import deque

pq = deque(
    sorted((assigment_costs[t][w], t, w) 
           for t in range(num_tasks) 
           for w in range(num_workers)
        )
    )

solution2 = set()
chosen_tasks = set()
chosen_workers = set()
total_cost = 0

while len(solution2) < 100:
    cost, t, w = pq.popleft()
    if t in chosen_tasks or w in chosen_workers:
        continue
    
    solution2.add((t, w))
    chosen_tasks.add(t)
    chosen_workers.add(w)
    total_cost += cost
    

In [ ]:
sum(assigment_costs[t][w] for t, w in solution2)

522

: 

Greedy doesn't work. Example

| | W1 | W2 |
|--|---|---|
|T1 | 1 | 2 |
|T2 | 3 | 100|

In [30]:
from collections import Counter

class Solution:
    def repeatLimitedString(self, s: str, repeatLimit: int) -> str:
        new_s = ''
        chars_sorted = sorted([(char, freq) for char, freq in Counter(s).items()])
        print(chars_sorted)
        
        onhold = None
        while chars_sorted:

            char, freq = chars_sorted.pop()
            
            if onhold and onhold[0] > char:
                take = 1
            else:
                take = min(repeatLimit, freq)

            print(char, take, onhold)
            new_s += char * take

            if onhold:
                chars_sorted.append(onhold)
                onhold = None

            if take < freq:
                onhold = (char, freq - take)
        
        return new_s
    

In [31]:
Solution().repeatLimitedString("robnsdvpuxbapuqgopqvxdrchivlifeepy", 2)

[('a', 1), ('b', 2), ('c', 1), ('d', 2), ('e', 2), ('f', 1), ('g', 1), ('h', 1), ('i', 2), ('l', 1), ('n', 1), ('o', 2), ('p', 4), ('q', 2), ('r', 2), ('s', 1), ('u', 2), ('v', 3), ('x', 2), ('y', 1)]
y 1 None
x 2 None
v 2 None
u 1 ('v', 1)
v 1 ('u', 1)
u 1 None
s 1 None
r 2 None
q 2 None
p 2 None
o 1 ('p', 2)
p 2 ('o', 1)
o 1 None
n 1 None
l 1 None
i 2 None
h 1 None
g 1 None
f 1 None
e 2 None
d 2 None
c 1 None
b 2 None
a 1 None


'yxxvvuvusrrqqppopponliihgfeeddcbba'

In [1]:
0 % 9

0